In [37]:
import pandas as pd 
import numpy as np 
from Bio import SeqIO
import src.tools.download
from src.tools import MMSeqs
from src.dataset import Dataset
from src.files import FASTAFile, GBFFFile
from src.reference import Reference
from src.clusterer import Clusterer
from tqdm import tqdm
import os
import json 
import seaborn as sns
from src import fillna 
import matplotlib.pyplot as plt 
from src.split import ClusterStratifiedShuffleSplit

%load_ext autoreload
%autoreload 2

# Because genomes are constantly being re-annotated, the GBFF files I downloaded a month or so ago are now not aligned with the 
# protein files I used to generate the most recent round of embeddings (05/23/2025). GBFF files were re-downloaded for 
# the Campylobacterota phylum on 05/25/2025. 

# ncbi = src.tools.download.NCBI()
# ncbi.get_genomes(genome_ids=genome_ids, include=['gbff'], dirs={'gbff':'../data/ncbi/gbffs/'})
# ncbi.cleanup()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
# Changing my training dataset strategy. Instead going to use the sequences directly from NCBI. 
genome_metadata_df = pd.read_csv('../data/genome_metadata.csv', index_col=0)
genome_ids = genome_metadata_df[genome_metadata_df.phylum == 'Campylobacterota'].index.values 


In [ ]:
# Want to build the dataset for Campylobacterota, but filter out the "suspect" sequences, i.e. those which are both hypothetical
# and have only ab initio evidence (i.e. no evidence of conservation). 

is_hypothetical = lambda df : df['product'] == 'hypothetical protein'
is_ab_initio = lambda df : df.evidence_type == 'ab initio prediction'
is_suspect = lambda df : is_hypothetical(df) & is_ab_initio(df) # This will be False for intergenic sequences. 

ncbi_df = list()
for genome_id in tqdm(genome_ids, desc='Building Campylobacterota dataset.'):
    protein_path = f'../data/ncbi/proteins/{genome_id}_protein.faa'
    gbff_path = f'../data/ncbi/gbffs/{genome_id}_genomic.gbff'

    gbff_df = GBFFFile(gbff_path).to_df()
    gbff_df = gbff_df[(gbff_df.feature == 'CDS') & (~gbff_df.pseudo)].copy()
    copy_numbers = gbff_df.protein_id.value_counts() # There are multiple copies of the same protein at different coordinates in the GBFF files with different coordinates. 
    gbff_df['copy_number'] = gbff_df.protein_id.map(copy_numbers)
    gbff_df = gbff_df.drop_duplicates('protein_id').copy()
    gbff_df = gbff_df.drop(columns=['seq']) # Use the sequences from the protein DataFrame, just to make sure everything is equal. 
    gbff_df = gbff_df.set_index('protein_id')
    gbff_df.index.name = 'id'

    protein_df = FASTAFile(path=protein_path).to_df(prodigal_output=False)
    protein_df = protein_df.drop(columns=['description'])

    assert len(protein_df) == len(gbff_df), 'Expected the number of non-pseudo CDS entries in the GBFF file to match the entries in the FASTA file.'
    assert np.all(np.sort(protein_df.index) == np.sort(gbff_df.index)), 'Expected the number of non-pseudo CDS entries in the GBFF file to match the entries in the FASTA file.'
    assert protein_df.index.is_unique and gbff_df.index.is_unique, 'Expected the indices of both DataFrames to be unique.'

    ncbi_df.append(protein_df.merge(gbff_df, left_index=True, right_index=True).assign(genome_id=genome_id))

ncbi_df = pd.concat(ncbi_df)

mask = is_suspect(ncbi_df)
print(f'Removing {mask.sum()} suspect sequences from the Campylobacterota dataset.')
ncbi_df = ncbi_df[~mask].copy()

ncbi_df.to_csv('../data/campylobacterota.csv')

Building Campylobacterota dataset.: 100%|██████████| 197/197 [06:16<00:00,  1.91s/it]


Removing 44475 suspect sequences from the Campylobacterota dataset.
Removing 341 sequences exceeding the maximum length from the Campylobacterota dataset.


In [ ]:
antifam_df = pd.read_csv('../data/antifam.csv', index_col=0)
ncbi_df = pd.read_csv('../data/campylobacterota.csv', index_col=0)

is_bacterial = lambda df : np.array(['2' in lineage.split(' ') for lineage in df.lineage])

mask = ~is_bacterial(antifam_df)
print(f'Removing {mask.sum()} non-bacterial sequences from the AntiFam dataset.')
antifam_df = antifam_df[~mask].copy()


Removing 9789 non-bacterial sequences from the AntiFam dataset.


In [ ]:
ncbi_df['library_entry_name'] = ncbi_df.genome_id 
ncbi_df['label'] = 1

antifam_df['library_entry_name'] = 'antifam'
antifam_df['label'] = 0

dataset_df = pd.concat([antifam_df, ncbi_df])
dataset_df = fillna(dataset_df, rules={str:'none', bool:False, int:0, float:0})

mask = (dataset_df.seq.apply(len) >= 2000) # Upper length bound is non-inclusive. 
print(f'Removing {mask.sum()} sequences exceeding the maximum length from the Campylobacterota dataset.')
dataset_df = dataset_df[~mask].copy()

def check_duplicate_ids_have_identical_sequences(dataset_df):
    duplicate_ids = dataset_df.index[dataset_df.index.duplicated()].unique()
    for id_ in tqdm(duplicate_ids, desc='check_duplicate_ids_have_identical_sequences'):
        seqs = dataset_df.seq[dataset_df.index == id_]
        assert np.all(seqs == seqs.iloc[0]), f'check_duplicate_ids_have_identical_sequences: Sequences with ID {id_} are not equal.'

# check_duplicate_ids_have_identical_sequences(dataset_df)

mask = dataset_df.seq.duplicated(keep='first')
print(f'Removing {mask.sum()} duplicate sequences from the Campylobacterota dataset')
dataset_df = dataset_df[~mask].copy()

dataset_df.to_csv('../data/dataset.csv')


Removing 16 sequences exceeding the maximum length from the Campylobacterota dataset.
Removing 9522 duplicate sequences from the Campylobacterota dataset


In [207]:
dataset_subset_df = dataset_df.sample(5000, random_state=42)
dataset_subset_df.to_csv('../data/dataset_subset.csv')

In [ ]:
# What is a good estimate for the number of clusters? Maybe use 50 percent sequence similarity?
mmseqs = MMSeqs()
mmseqs_cluster_df = mmseqs.cluster(dataset_df, job_name='cluster', output_dir='../data', sequence_identity=0.5, overwrite=False)
mmseqs.cleanup()
print('Number of clusters:', dataset_df.cluster_label.nunique())

In [ ]:
def train_test_split(dataset_path:str='../data/dataset_dereplicated.csv', cluster_path:str='../data/dataset_dereplicated_cluster.csv'):
    dataset = Dataset.from_csv(dataset_path, feature_type=None)
    splits = ClusterStratifiedShuffleSplit(dataset, cluster_path=cluster_path, n_splits=1)
    dataset_train, dataset_test = list(splits)[0]
    dataset_train.to_csv('../data/dataset_train.csv', metadata=True)
    dataset_test.to_csv('../data/dataset_test.csv', metadata=True)

    return dataset_train.to_df(metadata=True), dataset_test.to_df(metadata=True)

# train_test_split()

ClusterStratifiedShuffleSplit._split_non_homogenous_clusters: Found 538 non-homogenous clusters.
ClusterStratifiedShuffleSplit._load_clusters: Found 2001 singleton clusters with "real" labels.
ClusterStratifiedShuffleSplit._load_clusters: Found 422 singleton clusters with "spurious" labels.
ClusterStratifiedShuffleSplit.__init__: Adjusted training and test sizes are 0.801, 0.199.


In [39]:
prodigal_df = []
for genome_id in genome_ids:
    ref_df = Reference.load_ref(f'../data/ref/{genome_id}_ref.csv')
    ref_df['genome_id'] = genome_id
    prodigal_df.append(ref_df)
prodigal_df = pd.concat(prodigal_df)
prodigal_df.to_csv('../data/prodigal.csv')



In [ ]:
# def get_cluster_info(dataset_df:pd.DataFrame) -> pd.DataFrame:

#     df = list()
#     for cluster_label, cluster_df in dataset_df.groupby('cluster_label'):
#         cluster_dists = cluster_df.distance_to_cluster_center
#         row = dict()
#         row['cluster_label'] = cluster_label
#         row['max_distance_to_cluster_center'] = cluster_dists.max()
#         row['mean_distance_to_cluster_center'] = cluster_dists.mean()
#         row['min_distance_to_cluster_center'] = cluster_dists.min()
#         row['size'] = len(cluster_df)
#         row['homogenous'] = (cluster_df.label.nunique() == 1)

#         n_real, n_spurious = (cluster_df.label == 1).sum(), (cluster_df.label == 0).sum()
#         row['n_spurious'] = n_spurious
#         row['n_real'] = n_real
#         # row['majority_label'] = np.nan if (n_real == n_spurious) else int(n_real > n_spurious)
#         row['majority_label'] = int(n_real >= n_spurious)

#         if not row['homogenous']:
#             majority_label = row['majority_label']
#             minority_label = int(not majority_label)
#             row['mean_distance_to_cluster_center_majority_label'] = cluster_df.distance_to_cluster_center[cluster_df.label == majority_label].mean()
#             row['mean_distance_to_cluster_center_minority_label'] = cluster_df.distance_to_cluster_center[cluster_df.label == minority_label].mean()        
#         df.append(row)
#     df = pd.DataFrame(df).set_index('cluster_label')
#     return df